In [ ]:
import csv
import itertools
import operator
import numpy as np
import nltk
import sys
from datetime import datetime
!pip3 install utils
from utils import *

import matplotlib.pyplot as plt
%matplotlib inline
# Download NLTK model data (you need to do this once)
nltk.download("book")

In [ ]:
vocabulary_size = 8000
unknown_token = "UNKNOWN_TOKEN"
sentence_start_token = "SENTENCE_START"
sentence_end_token = "SENTENCE_END"

# Read the data and append SENTENCE_START and SENTENCE_END tokens
print("Reading CSV file")
with open('reddit-comments-2015-08.csv', 'r', encoding='latin1') as f:
    reader = csv.reader((line.replace('\0','') for line in f), skipinitialspace=True)
    next(reader)
    # Split full comments into sentences
    sentences = itertools.chain(*[nltk.sent_tokenize(x[0].lower()) for x in reader])
    # Append SENTENCE_START and SENTENCE_END
    sentences = ["%s %s %s" % (sentence_start_token, x, sentence_end_token) for x in sentences]
print("Parsed %d sentences." % (len(sentences)))
    
# Tokenize the sentences into words
tokenized_sentences = [nltk.word_tokenize(sent) for sent in sentences]

# Count the word frequencies
word_freq = nltk.FreqDist(itertools.chain(*tokenized_sentences))
print("Found %d unique words tokens." % len(word_freq.items()))

# Get the most common words and build index_to_word and word_to_index vectors
vocab = word_freq.most_common(vocabulary_size-1)
index_to_word = [x[0] for x in vocab]
index_to_word.append(unknown_token)
word_to_index = dict([(w,i) for i,w in enumerate(index_to_word)])

print("Using vocabulary size %d." % vocabulary_size)
print("The least frequent word in our vocabulary is '%s' and appeared %d times." % (vocab[-1][0], vocab[-1][1]))

# Replace all words not in our vocabulary with the unknown token
for i, sent in enumerate(tokenized_sentences):
    tokenized_sentences[i] = [w if w in word_to_index else unknown_token for w in sent]

print("\nExample sentence: '%s'" % sentences[1])
print("\nExample sentence after Pre-processing: '%s'" % tokenized_sentences[1])

In [ ]:
X_train = np.asarray([[word_to_index[w] for w in sent[:-1]] for sent in tokenized_sentences])
y_train = np.asarray([[word_to_index[w] for w in sent[1:]] for sent in tokenized_sentences])
x_example, y_example = X_train[18], y_train[18]
print ("x:\n%s\n%s" % (" ".join([index_to_word[x] for x in x_example]), x_example))
print ("\ny:\n%s\n%s" % (" ".join([index_to_word[x] for x in y_example]), y_example))

x:
SENTENCE_START what are n't you understanding about this ? !
[0, 52, 28, 16, 10, 853, 54, 26, 35, 70]

y:
what are n't you understanding about this ? ! SENTENCE_END
[52, 28, 16, 10, 853, 54, 26, 35, 70, 1]


In [ ]:
class RNNNumpy:
    
    def __init__(self, word_dim, hidden_dim=100, bptt_truncate=4):
        # Assign instance variables
        self.word_dim = word_dim
        self.hidden_dim = hidden_dim
        self.bptt_truncate = bptt_truncate
        # Randomly initialize the network parameters
        self.U = np.random.uniform(-np.sqrt(1./word_dim), np.sqrt(1./word_dim), (hidden_dim, word_dim))
        self.V = np.random.uniform(-np.sqrt(1./hidden_dim), np.sqrt(1./hidden_dim), (word_dim, hidden_dim))
        self.W = np.random.uniform(-np.sqrt(1./hidden_dim), np.sqrt(1./hidden_dim), (hidden_dim, hidden_dim))

def softmax(x):
    xt = np.exp(x - np.max(x))
    return xt / np.sum(xt)

def forward_propagation(self, x):
    # The total number of time steps
    T = len(x)
    # During forward propagation we save all hidden states in s because need them later.
    # We add one additional element for the initial hidden, which we set to 0
    s = np.zeros((T + 1, self.hidden_dim))
    s[-1] = np.zeros(self.hidden_dim)
    # The outputs at each time step. Again, we save them for later.
    o = np.zeros((T, self.word_dim))
    # For each time step...
    for t in np.arange(T):
        # Note that we are indxing U by x[t]. This is the same as multiplying U with a one-hot vector.
        s[t] = np.tanh(self.U[:,x[t]] + self.W.dot(s[t-1]))
        o[t] = softmax(self.V.dot(s[t]))
    return [o, s]

RNNNumpy.forward_propagation = forward_propagation

def predict(self, x):
    # Perform forward propagation and return index of the highest score
    o, s = self.forward_propagation(x)
    return np.argmax(o, axis=1)

RNNNumpy.predict = predict

np.random.seed(10)
model = RNNNumpy(vocabulary_size)
o, s = model.forward_propagation(X_train[10])
print(o.shape)
print(o)
predictions = model.predict(X_train[10])
print(predictions.shape)
print(predictions)


(16, 8000)
[[0.00012408 0.0001244  0.00012603 ... 0.00012515 0.00012488 0.00012508]
 [0.00012448 0.00012615 0.00012402 ... 0.00012514 0.00012425 0.00012528]
 [0.00012466 0.00012539 0.00012453 ... 0.0001251  0.00012575 0.00012581]
 ...
 [0.00012505 0.00012493 0.00012428 ... 0.00012581 0.00012497 0.00012489]
 [0.00012491 0.0001252  0.00012491 ... 0.00012442 0.00012416 0.0001257 ]
 [0.00012462 0.0001246  0.00012513 ... 0.00012453 0.00012502 0.00012456]]
(16,)
[1284 6751 6465 1387 2613 6546 4395 6455 7299 6722 6892 5354 6119 5401
 2147 5027]


In [ ]:
'''
the loss is defined as
L(y, o) = -\frac{1}{N} \sum_{n \in N} y_n log(o_n)
'''
def calculate_total_loss(self, x, y):
    L = 0
    # for each sentence ...
    for i in np.arange(len(y)):
        o, s = self.forward_propagation(x[i])
        # we only care about our prediction of the "correct" words
        correct_word_predictions = o[np.arange(len(y[i])), y[i]]
        # add to the loss based on how off we were
        L += -1 * np.sum(np.log(correct_word_predictions))
    return L

def calculate_loss(self, x, y):
    # divide the total loss by the number of training examples
    N = np.sum((len(y_i) for y_i in y))
    return self.calculate_total_loss(x, y)/N

RNNNumpy.calculate_total_loss = calculate_total_loss
RNNNumpy.calculate_loss = calculate_loss

print("Expected Loss for random prediction: %f" % np.log(vocabulary_size))
print("Actual loss: %f" % model.calculate_loss(X_train[:1000], y_train[:1000]))

Expected Loss for random prediction: 8.987197


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:18: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.


Actual loss: 8.987368


In [ ]:
## 3. BPTT
'''
1. we nudge the parameters into a direction that reduces the error. the direction is given by the gradient of the loss: \frac{\partial L}{\partial U}, 
\frac{\partial L}{\partial V}, \frac{\partial L}{\partial W}
2. we also need learning rate: which indicated how big of a step we want to make in each direction
Q: how to optimize SGD using batching, parallelism and adaptive learning rates.

RNN BPTT: because the parameters are shared by all time steps in the network, the gradient at each output depends not only on the calculations of the
current time step, but also the previous time steps.
'''
def bptt(self, x, y):
    T = len(y)
    # perform forward propagation
    o, s = self.forward_propagation(x)
    # we will accumulate the gradients in these variables
    dLdU = np.zeros(self.U.shape)
    dLdV = np.zeros(self.V.shape)
    dLdW = np.zeros(self.W.shape)
    delta_o = o
    delta_o[np.arange(len(y)), y] -= 1   # it is y_hat - y
    # for each output backwards ...
    for t in np.arange(T):
        dLdV += np.outer(delta_o[t], s[t].T)# at time step t, shape is word_dim * hidden_dim
        # initial delta calculation
        delta_t = self.V.T.dot(delta_o[t]) * (1 - (s[t] ** 2))
        # backpropagation through time (for at most self.bptt_truncate steps)
        # given time step t, go back from time step t, to t-1, t-2, ...
        for bptt_step in np.arange(max(0, t-self.bptt_truncate), t+1)[::-1]:
            # print("Backprogation step t=%d bptt step=%d" %(t, bptt_step))
            dLdW += np.outer(delta_t, s[bptt_step - 1])
            dLdU[:, x[bptt_step]] += delta_t
            # update delta for next step
            dleta_t = self.W.T.dot(delta_t) * (1 - s[bptt_step-1]**2)
    return [dLdU, dLdV, dLdW]

RNNNumpy.bptt = bptt

In [ ]:
## 4. SGD implementation
'''
two step:
1. calculate the gradients and perform the updates for one batch
2. loop through the training set and adjust the learning rate
'''
### 4.1. perform one step of SGD
def numpy_sgd_step(self, x, y, learning_rate):
    dLdU, dLdV, dLdW = self.bptt(x, y)
    self.U -= learning_rate * dLdU
    self.V -= learning_rate * dLdV
    self.W -= learning_rate * dLdW
RNNNumpy.sgd_step = numpy_sgd_step

### 4.2. outer SGD loop
'''
 - model: 
 - X_train:
 - y_train:
 - learning_rate:
 - nepoch:
 - evaluate loss_after:
'''
def train_with_sgd(model, X_train, y_train, learning_rate = 0.005, nepoch = 100, evaluate_loss_after = 5):
    # keep track of the losses so that we can plot them later
    losses = []
    num_examples_seen = 0
    for epoch in range(nepoch):
        # optionally evaluate the loss
        if (epoch % evaluate_loss_after == 0):
            loss = model.calculate_loss(X_train, y_train)
            losses.append((num_examples_seen, loss))
            time = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
            print("%s: loss after num_examples_seen=%d epoch=%d: %f" %(time, num_examples_seen, epoch, loss))
            # adjust the learning rate if loss increases
            if (len(losses) > 1 and losses[-1][1] > losses[-2][1]):
                learning_rate = learning_rate * 0.5
                print("setting learning rate to %f" %(learning_rate))
            sys.stdout.flush()
        # for each training example...
        for i in range(len(y_train)):
            # one sgd step
            model.sgd_step(X_train[i], y_train[i], learning_rate)
            #print("model.U", model.U)
            num_examples_seen += 1

np.random.seed(10)
model = RNNNumpy(vocabulary_size)
%timeit model.sgd_step(X_train[10], y_train[10], 0.005)

10 loops, best of 3: 87.4 ms per loop


In [ ]:
np.random.seed(10)
model = RNNNumpy(vocabulary_size)
print(model.U)
losses = train_with_sgd(model, X_train[1:100], y_train[1:100], nepoch = 10, evaluate_loss_after = 1)

[[ 0.00606691 -0.01071631  0.00298847 ... -0.01015375 -0.00477044
   0.00578017]
 [ 0.00944173 -0.0074726   0.00688785 ... -0.01038716 -0.00536007
  -0.00084724]
 [ 0.00682407 -0.00349747  0.0099399  ...  0.00208931  0.00112797
  -0.00020396]
 ...
 [-0.00758435  0.0015517  -0.01051649 ...  0.00565983 -0.00919381
  -0.00272508]
 [ 0.0014529   0.00681475  0.00820687 ...  0.00290203 -0.00640613
   0.00775899]
 [-0.0090378  -0.00120979  0.00874173 ... -0.00166648 -0.00791153
  -0.00890975]]


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:18: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.


2020-11-05 05:15:42: loss after num_examples_seen=0 epoch=0: 8.987353
2020-11-05 05:15:53: loss after num_examples_seen=99 epoch=1: 8.973855
2020-11-05 05:16:05: loss after num_examples_seen=198 epoch=2: 8.943834
2020-11-05 05:16:16: loss after num_examples_seen=297 epoch=3: 6.868403
2020-11-05 05:16:28: loss after num_examples_seen=396 epoch=4: 6.302610
2020-11-05 05:16:39: loss after num_examples_seen=495 epoch=5: 6.049862
2020-11-05 05:16:50: loss after num_examples_seen=594 epoch=6: 5.896365
2020-11-05 05:17:02: loss after num_examples_seen=693 epoch=7: 5.790338
2020-11-05 05:17:13: loss after num_examples_seen=792 epoch=8: 5.712210
2020-11-05 05:17:24: loss after num_examples_seen=891 epoch=9: 5.651878


In [ ]:
print(model.U)
def generate_sentence(model):
    # We start the sentence with the start token
    new_sentence = [word_to_index[sentence_start_token]]
    # Repeat until we get an end token
    while not new_sentence[-1] == word_to_index[sentence_end_token]:
        next_word_probs,s = model.forward_propagation(new_sentence)
        sampled_word = word_to_index[unknown_token]
        # We don't want to sample unknown words
        while sampled_word == word_to_index[unknown_token]:
            samples = np.random.multinomial(1, next_word_probs[-1])
            sampled_word = np.argmax(samples)
            print(sampled_word)
        new_sentence.append(sampled_word)
        if len(new_sentence) >= 20:
          break
    sentence_str = [index_to_word[x] for x in new_sentence[1:-1]]
    print(sentence_str)

num_sentences = 1
senten_min_length = 7

for i in range(num_sentences):
  generate_sentence(model)


[[ 0.16155555 -0.01071631  0.1092198  ... -0.01015375 -0.00477044
   0.11427511]
 [-0.01563373 -0.0074726  -0.07312643 ... -0.01038716 -0.00536007
   0.02745094]
 [-0.34838702 -0.00349747  0.07752491 ...  0.00208931  0.00112797
  -0.07091841]
 ...
 [-0.14211439  0.0015517  -0.11025854 ...  0.00565983 -0.00919381
  -0.05688476]
 [-0.45786422  0.00681475  0.05582244 ...  0.00290203 -0.00640613
  -0.05426409]
 [-0.49525312 -0.00120979  0.01973171 ... -0.00166648 -0.00791153
  -0.1019795 ]]
3271
6
73
16
7999
66
642
98
4828
16
4442
123
207
3720
2615
3
2
46
163
6436
['angle', 'i', 'no', "n't", 'will', 'allowed', 'good', 'transport', "n't", 'arts', 'into', 'used', 'prepare', 'frustrating', 'the', '.', 'at', 'our']
